In [20]:
import json
from gensim import corpora, models
import numpy as np

In [21]:
with open("train.json") as f:
    recipes = json.load(f)

In [22]:
print(recipes[1])

{'id': 25693, 'cuisine': 'southern_us', 'ingredients': ['plain flour', 'ground pepper', 'salt', 'tomatoes', 'ground black pepper', 'thyme', 'eggs', 'green tomatoes', 'yellow corn meal', 'milk', 'vegetable oil']}


In [23]:
import json
import pandas as pd

In [24]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [25]:
list_bad_ingr = list(filter(lambda x: dictionary.dfs[x] > 3500 and dictionary.dfs[x] < 3, dictionary.dfs.keys()))
dictionary.filter_tokens(list_bad_ingr)

In [26]:
np.random.seed(76543)
%time lda = models.ldamodel.LdaModel(corpus, num_topics=20, passes=40, id2word=dictionary, alpha=0.5, eta=1.25)

CPU times: user 17min 7s, sys: 3.32 s, total: 17min 10s
Wall time: 17min 13s


In [95]:
2**(-lda.log_perplexity(list(corpus)))

182.9532765470079

In [92]:
# 239

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [80]:
def create_row(pre_row):
    res = []
    for i in range(20):
        res.append(0)
        for obj in pre_row:
            if obj[0] == i:
                res[i] = obj[1]
                break
    return res  


x_data  = lda.get_document_topics(corpus)
x_data = list(map(lambda x: create_row(x), x_data))

In [81]:
x_label = list(map(lambda x: x['cuisine'], recipes))
dict_l = dict()
i = 0
for label in x_label:
    if not label in dict_l.keys():
        dict_l[label] = i
        i += 1
x_label = list(map(lambda x: dict_l[x], x_label))

In [82]:
a = np.arange(10)
a[:3]

array([0, 1, 2])

In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, grid_search

In [84]:
%%time
clf = LogisticRegression(C=100, penalty='l2', dual=False)
clf.fit(x_data[:35000], x_label[:35000])
print(metrics.classification_report(x_label[35000:], clf.predict(x_data[35000:])))

             precision    recall  f1-score   support

          0       0.42      0.24      0.31       147
          1       0.35      0.70      0.46       499
          2       0.48      0.13      0.20        95
          3       0.73      0.84      0.78       368
          4       0.50      0.06      0.10        52
          5       0.00      0.00      0.00       129
          6       0.62      0.83      0.71       918
          7       0.81      0.88      0.84       780
          8       0.42      0.86      0.57       335
          9       0.62      0.05      0.09       101
         10       0.54      0.24      0.33       181
         11       0.32      0.07      0.12        99
         12       0.45      0.34      0.39       208
         13       0.25      0.08      0.12        51
         14       0.35      0.11      0.17       300
         15       0.39      0.10      0.16       170
         16       0.00      0.00      0.00        91
         17       0.00      0.00      0.00   

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
lda.save('lda_20')